Import library

In [1]:
import nltk
import string
import time
import bisect
import pandas as pd
from nltk.corpus import words
from nltk.metrics.distance import (
    edit_distance,
    jaccard_distance,
    )
from nltk.util import ngrams
from trietree.trietree import loader
import pathlib
import sys


Get file path

In [2]:
file = pathlib.Path().parent.resolve()

Data Processing

In [3]:
correct_words = []
#Crete the list with only lower alphabet character
alphabet = list(string.ascii_letters)
#Read data
df = pd.read_csv(str(file) + r"\trietree\data\movies.csv")
unique_name = []
#put all data in a list
for name in df.title:
    unique_name.append(name.lower())
#remove duplicate
unique_name = list(dict.fromkeys(unique_name))
#get words from the data
for line in unique_name:
    for word in line.split():
        correct_words.append(word.lower())
for word in alphabet:
    correct_words.append(word)
#remove duplicate
correct_words = list(dict.fromkeys(correct_words))

correct_spellings = correct_words
spellings_series = pd.Series(correct_spellings)

B Tree

In [4]:
class _BNode(object):
    __slots__ = ["tree", "contents", "children"]

    def __init__(self, tree, contents=None, children=None):
        self.tree = tree
        self.contents = contents or []
        self.children = children or []
        if self.children:
            assert len(self.contents) + 1 == len(self.children), \
                    "one more child than data item required"

    def __repr__(self):
        name = getattr(self, "children", 0) and "Branch" or "Leaf"
        return "<%s %s>" % (name, ", ".join(map(str, self.contents)))

    def lateral(self, parent, parent_index, dest, dest_index):
        if parent_index > dest_index:
            dest.contents.append(parent.contents[dest_index])
            parent.contents[dest_index] = self.contents.pop(0)
            if self.children:
                dest.children.append(self.children.pop(0))
        else:
            dest.contents.insert(0, parent.contents[parent_index])
            parent.contents[parent_index] = self.contents.pop()
            if self.children:
                dest.children.insert(0, self.children.pop())

    def shrink(self, ancestors):
        parent = None

        if ancestors:
            parent, parent_index = ancestors.pop()
            # try to lend to the left neighboring sibling
            if parent_index:
                left_sib = parent.children[parent_index - 1]
                if len(left_sib.contents) < self.tree.order:
                    self.lateral(
                            parent, parent_index, left_sib, parent_index - 1)
                    return

            # try the right neighbor
            if parent_index + 1 < len(parent.children):
                right_sib = parent.children[parent_index + 1]
                if len(right_sib.contents) < self.tree.order:
                    self.lateral(
                            parent, parent_index, right_sib, parent_index + 1)
                    return

        center = len(self.contents) // 2
        sibling, push = self.split()

        if not parent:
            parent, parent_index = self.tree.BRANCH(
                    self.tree, children=[self]), 0
            self.tree._root = parent

        # pass the median up to the parent
        parent.contents.insert(parent_index, push)
        parent.children.insert(parent_index + 1, sibling)
        if len(parent.contents) > parent.tree.order:
            parent.shrink(ancestors)

    def grow(self, ancestors):
        parent, parent_index = ancestors.pop()

        minimum = self.tree.order // 2
        left_sib = right_sib = None

        # try to borrow from the right sibling
        if parent_index + 1 < len(parent.children):
            right_sib = parent.children[parent_index + 1]
            if len(right_sib.contents) > minimum:
                right_sib.lateral(parent, parent_index + 1, self, parent_index)
                return

        # try to borrow from the left sibling
        if parent_index:
            left_sib = parent.children[parent_index - 1]
            if len(left_sib.contents) > minimum:
                left_sib.lateral(parent, parent_index - 1, self, parent_index)
                return

        # consolidate with a sibling - try left first
        if left_sib:
            left_sib.contents.append(parent.contents[parent_index - 1])
            left_sib.contents.extend(self.contents)
            if self.children:
                left_sib.children.extend(self.children)
            parent.contents.pop(parent_index - 1)
            parent.children.pop(parent_index)
        else:
            self.contents.append(parent.contents[parent_index])
            self.contents.extend(right_sib.contents)
            if self.children:
                self.children.extend(right_sib.children)
            parent.contents.pop(parent_index)
            parent.children.pop(parent_index + 1)

        if len(parent.contents) < minimum:
            if ancestors:
                # parent is not the root
                parent.grow(ancestors)
            elif not parent.contents:
                # parent is root, and its now empty
                self.tree._root = left_sib or self

    def split(self):
        center = len(self.contents) // 2
        median = self.contents[center]
        sibling = type(self)(
                self.tree,
                self.contents[center + 1:],
                self.children[center + 1:])
        self.contents = self.contents[:center]
        self.children = self.children[:center + 1]
        return sibling, median

    def insert(self, index, item, ancestors):
        self.contents.insert(index, item)
        if len(self.contents) > self.tree.order:
            self.shrink(ancestors)

    def remove(self, index, ancestors):
        minimum = self.tree.order // 2

        if self.children:
            # try promoting from the right subtree first,
            # but only if it won't have to resize
            additional_ancestors = [(self, index + 1)]
            descendent = self.children[index + 1]
            while descendent.children:
                additional_ancestors.append((descendent, 0))
                descendent = descendent.children[0]
            if len(descendent.contents) > minimum:
                ancestors.extend(additional_ancestors)
                self.contents[index] = descendent.contents[0]
                descendent.remove(0, ancestors)
                return

            # fall back to the left child
            additional_ancestors = [(self, index)]
            descendent = self.children[index]
            while descendent.children:
                additional_ancestors.append(
                        (descendent, len(descendent.children) - 1))
                descendent = descendent.children[-1]
            ancestors.extend(additional_ancestors)
            self.contents[index] = descendent.contents[-1]
            descendent.remove(len(descendent.children) - 1, ancestors)
        else:
            self.contents.pop(index)
            if len(self.contents) < minimum and ancestors:
                self.grow(ancestors)

class BTree(object):
    BRANCH = LEAF = _BNode

    def __init__(self, order):
        self.order = order
        self._root = self._bottom = self.LEAF(self)
        self.film = []

    def _path_to(self, item):
        current = self._root
        ancestry = []

        while getattr(current, "children", None):
            index = bisect.bisect_left(current.contents, item)
            ancestry.append((current, index))
            if index < len(current.contents) \
                    and current.contents[index] == item:
                return ancestry
            current = current.children[index]

        index = bisect.bisect_left(current.contents, item)
        ancestry.append((current, index))
        present = index < len(current.contents)
        present = present and current.contents[index] == item

        return ancestry

    def _present(self, item, ancestors):
        last, index = ancestors[-1]
        return index < len(last.contents) and last.contents[index] == item

    def insert(self, item):
        current = self._root
        ancestors = self._path_to(item)
        node, index = ancestors[-1]
        while getattr(node, "children", None):
            node = node.children[index]
            index = bisect.bisect_left(node.contents, item)
            ancestors.append((node, index))
        node, index = ancestors.pop()
        node.insert(index, item, ancestors)

    def remove(self, item):
        current = self._root
        ancestors = self._path_to(item)

        if self._present(item, ancestors):
            node, index = ancestors.pop()
            node.remove(index, ancestors)
        else:
            raise ValueError("%r not in %s" % (item, self.__class__.__name__))

    def is_prefix(self, sub, s):
        l = sub.split()
        s1 = ""
        for i in range(len(l)):
            s1 = " ".join(l[:len(l) - i])
            if s1 == s[:len(s1)]:
                return True
        return False

    def search_film(self, k, x=None):
        if x is not None:
            i = 0
            while i<len(x.contents) and not self.is_prefix(k,x.contents[i].lower()) and k > x.contents[i].lower():
                i = i+1
            if i<len(x.contents) and self.is_prefix(k,x.contents[i].lower()):
                while i<len(x.contents) and self.is_prefix(k,x.contents[i].lower()):
                    self.film.append(x.contents[i])
                    if not len (x.children) == 0:
                        if i+1<len(x.contents): 
                            if not self.is_prefix(k,x.contents[i+1].lower()):
                                self.search_film(k, x.children[i])
                                if i+1 < len(x.children):
                                    self.search_film(k, x.children[i + 1])
                            else:
                                self.search_film(k, x.children[i])
                        else:
                            self.search_film(k, x.children[i])
                            if (i+1<len(x.children)):
                                self.search_film(k, x.children[i + 1])  
                    i = i + 1
            elif len(x.children) != 0 and i < len(x.children):
                self.search_film(k, x.children[i])
        else:
            return self.search_film(k, self._root)

    def is_sub(self, text, s):
        s = s.lower()
        return s.startswith(text)

    def search_2(self, k, x=None):
        if x is not None:
            i = 0
            while i<len(x.contents) and not self.is_sub(k,x.contents[i].lower()) and k > x.contents[i].lower():
                i = i+1
            if i<len(x.contents) and self.is_sub(k,x.contents[i].lower()):
                while i<len(x.contents) and self.is_sub(k,x.contents[i].lower()):
                    self.film.append(x.contents[i])
                    if not len (x.children) == 0:
                        if i+1<len(x.contents): 
                            if not self.is_sub(k,x.contents[i+1].lower()):
                                self.search_2(k, x.children[i])
                                if i+1 < len(x.children):
                                    self.search_2(k, x.children[i + 1])
                            else:
                                self.search_2(k, x.children[i])
                        else:
                            self.search_2(k, x.children[i])
                            if (i+1<len(x.children)):
                                self.search_2(k, x.children[i + 1])  
                    i = i + 1
            elif len(x.children) != 0 and i < len(x.children):
                self.search_2(k, x.children[i])
        else:
            return self.search_2(k, self._root)

def text_input_preprocess(text):
    return " ".join(text.lower().split())

R-B Tree

In [5]:
import sys

# data structure that represents a node in the tree
class Node():
    def __init__(self, data):
        self.data = data  # holds the key
        self.parent = None #pointer to the parent
        self.left = None # pointer to left child
        self.right = None #pointer to right child
        self.color = 1 # 1 . Red, 0 . Black


# class RedBlackTree implements the operations in Red Black Tree
class RedBlackTree():
    def __init__(self):
        self.TNULL = Node(0)
        self.TNULL.color = 0
        self.TNULL.left = None
        self.TNULL.right = None
        self.root = self.TNULL

    def __pre_order_helper(self,node,search,result):
        if len(result) <5:
            if node != None:
                if str(node.data).startswith(search):
                    result.append(str(node.data))
                self.__pre_order_helper(node.left,search,result)
                self.__pre_order_helper(node.right,search,result)

    def __in_order_helper(self, node):
        if node != None:
            self.__in_order_helper(node.left)
            sys.stdout.write(node.data + " ")
            self.__in_order_helper(node.right)

    def __post_order_helper(self, node):
        if node != None:
            self.__post_order_helper(node.left)
            self.__post_order_helper(node.right)
            sys.stdout.write(node.data + " ")

    def __search_tree_helper(self, node, key):
        if node == None or (str(node.data).startswith(key)):
            return node
        if str(key) < str(node.data):
            return self.__search_tree_helper(node.left, key)
        return self.__search_tree_helper(node.right, key)

    # fix the rb tree modified by the delete operation
    def __fix_delete(self, x):
        while x != self.root and x.color == 0:
            if x == x.parent.left:
                s = x.parent.right
                if s.color == 1:
                    # case 3.1
                    s.color = 0
                    x.parent.color = 1
                    self.left_rotate(x.parent)
                    s = x.parent.right

                if s.left.color == 0 and s.right.color == 0:
                    # case 3.2
                    s.color = 1
                    x = x.parent
                else:
                    if s.right.color == 0:
                        # case 3.3
                        s.left.color = 0
                        s.color = 1
                        self.right_rotate(s)
                        s = x.parent.right

                    # case 3.4
                    s.color = x.parent.color
                    x.parent.color = 0
                    s.right.color = 0
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                s = x.parent.left
                if s.color == 1:
                    # case 3.1
                    s.color = 0
                    x.parent.color = 1
                    self.right_rotate(x.parent)
                    s = x.parent.left

                if s.left.color == 0 and s.right.color == 0:
                    # case 3.2
                    s.color = 1
                    x = x.parent
                else:
                    if s.left.color == 0:
                        # case 3.3
                        s.right.color = 0
                        s.color = 1
                        self.left_rotate(s)
                        s = x.parent.left 

                    # case 3.4
                    s.color = x.parent.color
                    x.parent.color = 0
                    s.left.color = 0
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 0

    def __rb_transplant(self, u, v):
        if u.parent == None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def __delete_node_helper(self, node, key):
        # find the node containing key
        z = self.TNULL
        while node != self.TNULL:
            if node.data == key:
                z = node

            if node.data <= key:
                node = node.right
            else:
                node = node.left

        if z == self.TNULL:
            print("Couldn't find key in the tree")
            return

        y = z
        y_original_color = y.color
        if z.left == self.TNULL:
            x = z.right
            self.__rb_transplant(z, z.right)
        elif (z.right == self.TNULL):
            x = z.left
            self.__rb_transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                x.parent = y
            else:
                self.__rb_transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.__rb_transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
        if y_original_color == 0:
            self.__fix_delete(x)
    
    # fix the red-black tree
    def  __fix_insert(self, k):
        while k.parent.color == 1:
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left # uncle
                if u.color == 1:
                    # case 3.1
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent
                else:
                    if k == k.parent.left:
                        # case 3.2.2
                        k = k.parent
                        self.right_rotate(k)
                    # case 3.2.1
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.left_rotate(k.parent.parent)
            else:
                u = k.parent.parent.right # uncle

                if u.color == 1:
                    # mirror case 3.1
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent 
                else:
                    if k == k.parent.right:
                        # mirror case 3.2.2
                        k = k.parent
                        self.left_rotate(k)
                    # mirror case 3.2.1
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.right_rotate(k.parent.parent)
            if k == self.root:
                break
        self.root.color = 0

    def __print_helper(self, node, indent, last):
        # print the tree structure on the screen
        if node != self.TNULL:
            sys.stdout.write(indent)
            if last:
                sys.stdout.write("R----")
                indent += "     "
            else:
                sys.stdout.write("L----")
                indent += "|    "

            s_color = "RED" if node.color == 1 else "BLACK"
            print(str(node.data) + "(" + s_color + ")")
            self.__print_helper(node.left, indent, False)
            self.__print_helper(node.right, indent, True)
    
    # Pre-Order traversal
    # Node.Left Subtree.Right Subtree
    def preorder(self,search,result):
        self.__pre_order_helper(self.root,search,result)

    # In-Order traversal
    # left Subtree . Node . Right Subtree
    def inorder(self):
        self.__in_order_helper(self.root)

    # Post-Order traversal
    # Left Subtree . Right Subtree . Node
    def postorder(self):
        self.__post_order_helper(self.root)

    # search the tree for the key k
    # and return the corresponding node
    def searchTree(self,node , k):
        return self.__search_tree_helper(node, k)

    def continueSearch(self, node, k):
        if node == None:
            node1 = None
            node2 = None
            return node1,node2
        if (node.right and node.left) != None:
            node1 = self.__search_tree_helper(node.right, k)
            node2 = self.__search_tree_helper(node.left, k)
            return node1,node2
        elif node.right != None and node.left == None:
            node1 = self.__search_tree_helper(node.right, k)
            node2 = None
            return node1,node2
        elif node.right == None and node.left != None:
            node1 = None
            node2 = self.__search_tree_helper(node.left, k)
            return node1,node2
        else:
            node1 = None
            node2 = None
            return node1,node2
    # find the node with the minimum key
    def minimum(self, node):
        while node.left != self.TNULL:
            node = node.left
        return node

    # find the node with the maximum key
    def maximum(self, node):
        while node.right != self.TNULL:
            node = node.right
        return node

    # find the successor of a given node
    def successor(self, x):
        # if the right subtree is not None,
        # the successor is the leftmost node in the
        # right subtree
        if x.right != self.TNULL:
            return self.minimum(x.right)

        # else it is the lowest ancestor of x whose
        # left child is also an ancestor of x.
        y = x.parent
        while y != self.TNULL and x == y.right:
            x = y
            y = y.parent
        return y

    # find the predecessor of a given node
    def predecessor(self,  x):
        # if the left subtree is not None,
        # the predecessor is the rightmost node in the 
        # left subtree
        if (x.left != self.TNULL):
            return self.maximum(x.left)

        y = x.parent
        while y != self.TNULL and x == y.left:
            x = y
            y = y.parent

        return y

    # rotate left at node x
    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    # rotate right at node x
    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.TNULL:
            y.right.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y

    # insert the key to the tree in its appropriate position
    # and fix the tree
    def insert(self, key):
        # Ordinary Binary Search Insertion
        node = Node(key)
        node.parent = None
        node.data = key
        node.left = self.TNULL
        node.right = self.TNULL
        node.color = 1 # new node must be red

        y = None
        x = self.root

        while x != self.TNULL:
            y = x
            if node.data < x.data:
                x = x.left
            else:
                x = x.right

        # y is parent of x
        node.parent = y
        if y == None:
            self.root = node
        elif node.data < y.data:
            y.left = node
        else:
            y.right = node

        # if new node is a root node, simply return
        if node.parent == None:
            node.color = 0
            return

        # if the grandparent is None, simply return
        if node.parent.parent == None:
            return

        # Fix the tree
        self.__fix_insert(node)

    def get_root(self):
        return self.root

    # delete the node from the tree
    def delete_node(self, data):
        self.__delete_node_helper(self.root, data)

    # print the tree structure on the screen
    def pretty_print(self):
        self.__print_helper(self.root, "", True)


Jaccard method

In [6]:
def jaccard(entries, gram_number):
    
    outcomes = []
    for entry in entries: #iteratively for loop
        spellings = spellings_series[spellings_series.str.startswith(entry[0])] 
        distances = ((jaccard_distance(set(ngrams(entry, gram_number)),
                                       set(ngrams(word, gram_number))), word)
                     for word in spellings)
        closest = min(distances)
        outcomes.append(closest[1])
    return outcomes

JDreco method

In [7]:
def JDreco(entries=[]):
    return jaccard(entries, len(entries))

In [8]:
l = []
for i in df.title:
    l.append(i.lower())

Insert R-B Tree

In [9]:

bst = RedBlackTree()
for i in l:
    bst.insert(i)



Insert B tree with 3 keys

In [10]:
b = BTree(3)
for i in l:
    b.insert(i)

Insert B tree with 100 keys

In [11]:
b2 = BTree(100)
for i in l:
    b2.insert(i)

Trie Tree load and insert

In [12]:
tree = loader.initialize_fit_tree()

Search funcion with R-B Tree

In [13]:
def R_B_Search(bst_search):
    start_time = time.time()
    node = []
    result = []
    data = []
    node1 = None
    node2 = None
    search_node = bst.searchTree(bst.get_root(),bst_search)
    if search_node != bst.get_root() and search_node != None:
        result.append(str(search_node.data))
        node.append(search_node)    
    while(node):
        node1,node2 = bst.continueSearch(node[0],bst_search)
        if (node1 != None and node2 != None):
            result.append(str(node1.data))
            result.append(str(node2.data))
            node.append(node1)
            node.append(node2)
            node.remove(node[0])
        elif node1 != None and node2 == None:
            result.append(str(node1.data))
            node.append(node1)
            node.remove(node[0])
        elif node2 != None and node1 == None:
            result.append(str(node2.data))
            node.append(node2)
            node.remove(node[0])
        else:
            node.remove(node[0])
    end_time = time.time()
    elapsed_time = end_time - start_time
    result =list(dict.fromkeys(result))
    if len(result) > 5:
        for i in range(5):
            data.append(result[i])
    else:
        for i in range(len(result)):
            data.append(result[i])
    print ("R-B Tree: {0}".format(elapsed_time) + " seconds")
    return data, elapsed_time
    

Search funcion with Trie Tree

In [14]:
def Trie_Search(bst_search):
    output = tree.query(bst_search)
    result = output['result']
    result =list(dict.fromkeys(result))
    time = output['inference_time']
    print(f'Trie Tree: {time} seconds')
    return result, time

Search funcion with 3 keys B Tree

In [15]:
def B3_Search(bst_search):
    start_time = time.time()
    b.search_2(bst_search)
    end_time = time.time()
    elapsed_time = end_time - start_time
    b.film =list(dict.fromkeys(b.film))
    data = []
  
    if len(b.film) > 5:
        for i in range(5):
            data.append(b.film[i])
    else:
        for i in range(len(b.film)):
            data.append(b.film[i])
    
    b.film.clear()
    
    
    print ("3 keys B Tree: {0}".format(elapsed_time) + " seconds")

    return data, elapsed_time
    

Search funcion with 100 keys B Tree

In [16]:
def B100_Search(bst_search):
    start_time2 = time.time()
    b2.search_2(bst_search)
    end_time2 = time.time()
    b2.film =list(dict.fromkeys(b2.film))
    data = []
    elapsed_time2 = end_time2 - start_time2
    if len(b2.film) > 5:
        for i in range(5):
            data.append(b2.film[i])
    else:
        for i in range(len(b2.film)):
            data.append(b2.film[i])
    b2.film.clear()
    
    
    print ("100 keys B Tree: {0}".format(elapsed_time2) + " seconds")

    return data, elapsed_time2

Data count

In [17]:
df.shape

(5000, 11)

Searching UI

In [26]:
from tkinter import *

root = Tk()
root.title("Search Engine")
root.geometry("500x550")

def update(data):
    my_list.delete(0, END)

    for item in data:
        my_list.insert(END, item)

def fillout(e):
    my_entry.delete(0, END)
    my_entry.insert(END,my_list.get(ANCHOR))

def checkbr():
    typed = my_entry.get()

    if typed == '':
        data = movie
    else:
        user_list = typed.split()
        user_correct = JDreco(user_list)
        bst_search = (' '.join(str(x) for x in user_correct))
        data = []
        data, elapsed_time = R_B_Search(bst_search)
        my_label2.config(text="Time: {0}".format(elapsed_time) + " seconds")
    update(data)
def checkb3():
    typed = my_entry.get()

    if typed == '':
        data = movie
    else:
        user_list = typed.split()
        user_correct = JDreco(user_list)
        bst_search = (' '.join(str(x) for x in user_correct))
        data = []
        data, elapsed_time = B3_Search(bst_search)
        my_label2.config(text="Time: {0}".format(elapsed_time) + " seconds")
    update(data)

def checkb100():
    typed = my_entry.get().lower()

    if typed == '':
        data = movie
    else:
        user_list = typed.split()
        user_correct = JDreco(user_list)
        bst_search = (' '.join(str(x) for x in user_correct))
        data = []
        data, elapsed_time = B100_Search(bst_search)
        my_label2.config(text="Time: {0}".format(elapsed_time) + " seconds")
    update(data)

def checktr():
    typed = my_entry.get().lower()

    if typed == '':
        data = movie
    else:
        user_list = typed.split()
        user_correct = JDreco(user_list)
        bst_search = (' '.join(str(x) for x in user_correct))
        data = []
        data, elapsed_time = Trie_Search(bst_search)
        my_label2.config(text="Time: {0}".format(elapsed_time) + " seconds")
    update(data)


my_label = Label(root, text="Start Typing...", font=("Helvetica", 14), fg="grey")

my_label.pack(pady=20)

global my_label2
my_label2 = Label(root, text="Time: 0.0", font=("Helvetica", 14), fg="black")

my_label2.pack(pady=20)


my_entry = Entry(root, font=("Helvetica", 20))
my_entry.pack()

my_list = Listbox(root, width=50)
my_list.pack(pady=40)

my_button = Button(root, text="Search with Red-Black Tree", command=checkbr)
my_button.pack()

my_button2 = Button(root, text="Search with B Tree with 3 key", command=checkb3)
my_button2.pack()

my_button3 = Button(root, text="Search with B Tree with 100 key", command=checkb100)
my_button3.pack()

my_button4 = Button(root, text="Search with Trie Tree", command=checktr)
my_button4.pack()

movie = []
for item in df.title:
    movie.append(str(item).lower())

update(movie)

my_list.bind("<<ListboxSelect>>", fillout)

root.mainloop()

R-B Tree: 0.0 seconds
3 keys B Tree: 0.0 seconds
100 keys B Tree: 0.0 seconds
Trie Tree: 0.0003666877746582031 seconds
